In [1]:
from sage.matroids.set_system import SetSystem;

def unfrozenSetSystem(S: SetSystem):
    return [set(s) for s in S];
def compareSetSystems(S: SetSystem, S2: SetSystem):
    for A in S:
        if not any(A == B for B in S2):
            return False;
    for B in S2:
        if not any(A == B for A in S):
            return False;
    return True;
        
class TransversalMatroid:
    def __init__(self, n: int, sets: list):
        for s in sets:
            if len(sets)!=1 and len(s)==0:
                sets.remove(s);
        self.n = n;
        self.sets = SetSystem(range(1, n+1), sets);
        self.maxPresentation = None;
        self.rank = None;
    def __str__(self):
        return str([set(s) for s in self.sets]);
    def __eq__(self, other):
        return self.n == other.n and compareSetSystems(self.sets, other.sets);
    def copy(self):
        return TransversalMatroid(self.n, self.sets);
    def addToPosition(self, i: int, e: int):
        aux = unfrozenSetSystem(self.sets);
        aux[i].add(e);
        return TransversalMatroid(self.n, aux);
    def removeElement(self, e: int):
        aux = unfrozenSetSystem(self.sets);
        for a in aux:
            a.discard(e);
            if len(a) == 0:
                aux.remove(a);
        return TransversalMatroid(self.n, aux);
    def removeSet(self, i: int):
        aux = [];
        for j, A in enumerate(self.sets):
            if i != j:
                aux.append(A);
        return TransversalMatroid(self.n, aux);
    def deletion(self, X: set):
        aux = unfrozenSetSystem(self.sets);
        for i, a in enumerate(aux):
            aux[i] = a - X;
        for a in aux:
            if len(a)==0:
                aux.remove(a);
        return TransversalMatroid(self.n, aux);
    def getRank(self):
        if self.rank:
            return self.rank;
        groundset = Set(range(1, self.n + 1));
        r = len(self.sets);
        while r > 0:
            for candidate in groundset.subsets(r):
                if isIndependent(self.n, self, candidate):
                    return r;
            r = r - 1;
        return 0;
    def getMaxPresentation(self):
        if self.maxPresentation:
            return self.maxPresentation;
        p = TransversalMatroid(self.n, self.sets);
        for i, A in enumerate(p.sets):
            for e in range(1, self.n+1):
                if e not in A:
                    q = p.removeSet(i);
                    q = q.deletion(A);
                    if isColoop(self.n, q, e):
                        p = p.addToPosition(i, e);
        return p;
    def extension(self, I: set):
        aux = unfrozenSetSystem(self.sets);
        for i in I:
            aux[i].add(self.n + 1);
        return TransversalMatroid(self.n + 1, aux);
    def addColoop(self):
        aux = unfrozenSetSystem(self.sets);
        aux.append({self.n + 1});
        return TransversalMatroid(self.n + 1, aux);
    def relabel(self, phi: Permutation):
        aux = unfrozenSetSystem(self.sets);
        newSets = [];
        for a in aux:
            newSets.append([phi(el) for el in a]);
        return TransversalMatroid(self.n, newSets);
    def canonicalOrder(self):
        aux = unfrozenSetSystem(self.sets);
        newSets = [];
        for a in aux:
            newSets.append([phi(el) for el in a]);
        return TransversalMatroid(self.n, newSets);

In [51]:
# UNIT TESTS

# Auxiliar functions
n = 5;
p = TransversalMatroid(n, [[1,2,3], [1,2,3,4], [1, 2, 3, 4]]);
assert isLoop(n, p, 5) == True;
assert isLoop(n, p, 4) == False;
print('OK: isLoop');


n = 4;
p = TransversalMatroid(n, [[1,2], [1,2], [3, 4]]);
assert isIndependent(n, p, [1,4]) == True;
assert isIndependent(n, p, [3,4]) == False;
print('OK: isIndependent');


n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1, 2, 3, 4]]);
assert isColoop(n, p, 1) == True;
assert isColoop(n, p, 2) == False;
print('OK: isColoop');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1, 2, 3, 4]]);
assert {1,2,3} in getBases(n, p);
assert not {2,3,4} in getBases(n, p);
print('OK: getBases');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1, 2, 3, 4]]);
assert {1,2,3} in getBases(n, p);
assert not {2,3,4} in getBases(n, p);
print('OK: getBases');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
X = frozenset({1});
Y = frozenset({2,3});
assert support(p, X) == 3;
assert support(p, Y) == 1;
print('OK: support');


# Presentation class internal functions
n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
q = p.addToPosition(0, 4);
assert p.sets[0] == frozenset({1});
assert q.sets[0] == frozenset({1, 4});
print('OK: addToPosition');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
q = p.removeElement(2);
assert p.sets[1] == frozenset({1, 2, 3, 4});
assert q.sets[1] == frozenset({1, 3, 4});
t = p.removeElement(1);
assert len(p.sets) == 3;
assert len(t.sets) == 2;
print('OK: removeElement');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
q = p.removeSet(2);
assert len(p.sets) == 3;
assert len(q.sets) == 2;
assert q.sets[0] == frozenset({1});
assert q.sets[1] == frozenset({1,2,3,4});
print('OK: removeSet');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
X = {1, 2};
q = p.deletion(X);
assert len(q.sets) == 2;
assert q.sets[0] == frozenset({3, 4});
assert q.sets[1] == frozenset({4});
print('OK: deletion');

n=7;
p = TransversalMatroid(n, [[1,2,3,4,5,7], [1,2,3,4,5], [4,5,6,7]]);
q = p.getMaxPresentation();
assert q.sets[0] == frozenset({1,2,3,4,5,6,7}) and q.sets[1] == frozenset({1,2,3,4,5,6,7}) and q.sets[2] == frozenset({4,5,6,7})
print('OK: findMaxPresentation');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
q = p.extension({0,2});
assert len(q.sets) == 3;
assert q.sets[0] == frozenset({1,n+1}) and q.sets[1] == p.sets[1] and q.sets[2] == frozenset({1,4,n+1})
print('OK: extension');

n=4;
p = TransversalMatroid(n, [[1], [1,2,3,4], [1,4]]);
q = p.addColoop();
assert len(q.sets) == 4;
assert q.sets[0] == p.sets[0] and q.sets[1] == p.sets[1] and q.sets[2] == p.sets[2] and q.sets[3] == frozenset({n+1});
print('OK: addColoop');


OK: isLoop
OK: isIndependent
OK: isColoop
OK: getBases
OK: getBases
OK: support
OK: addToPosition
OK: removeElement
OK: removeSet
OK: deletion
OK: findMaxPresentation
OK: extension
OK: addColoop


In [2]:
# Auxiliary functions
def support(p: TransversalMatroid, X: frozenset):
    count = 0;
    for A in p.sets:
        if len(A.intersection(X))>0:
            count = count + 1;
    return count;

def isIndependent(n: int, p: TransversalMatroid, X):
    for sub in Set(X).subsets():
        if support(p, sub) < len(sub):
            return False;
    return True;

def isLoop(n: int, p: TransversalMatroid, e: int):
    for A in p.sets:
        if e in A:
            return False;
    return True;
def isColoop(n: int, p: TransversalMatroid, e: int):
    for b in getBases(n, p):
        if not e in b:
            return False;
    return True;

def getBases(n: int, p: TransversalMatroid):
    groundset = Set(range(1, n+1));
    r = p.getRank();
    bases = [];
    for candidate in groundset.subsets(r):
        if isIndependent(n, p, candidate):
            bases.append(frozenset(candidate));
    return bases;
def isomorphic(p1: TransversalMatroid, p2: TransversalMatroid):
    key = hash(str(p1))+hash(str(p2));
    if key in computedIsomorphic:
        return computedIsomorphic[key];

    if p1.n != p2.n:
        computedIsomorphic[key] = False;
        return False;
    groundset = Set(range(1, p1.n+1));
    M1 = Matroid(groundset, bases=getBases(p1.n, p1));
    M2 = Matroid(groundset, bases=getBases(p2.n, p2));
    
    result = M1.is_isomorphic(M2);
    computedIsomorphic[key] = result;
    return result;


In [11]:
def isDuplicate(stored: list, new: TransversalMatroid):
    relabeledNewList = [new.relabel(phi) for phi in Permutations(new.n)];
    for p in stored:
        key = str(p)+str(new);
        if key in relabeled and relabeled[key] == True:
            return True;
        if key in relabeled and relabeled[key] == False:
            continue;
        if p.n == new.n and len(p.sets) == len(new.sets) and any(relabeledNew == p for relabeledNew in relabeledNewList):
            relabeled[key] = True;
            return True;
        else:
            relabeled[key] = False;
    return False;
def getMaxPresentations(N: int, minPres: list):
    realMaxPres = [[[] for i in range(1, N+2)] for i in range(1, N+2)];
    for n in range(1, N+1):
        for i,p in enumerate(minPresentations[n]):
            found = false;
            r = p.getRank();
            maxP = p.getMaxPresentation();
            for q in realMaxPres[n][r]:
                if isomorphic(maxP, q):
                    found = true;
                    break;
            if not found:
                realMaxPres[n][r].append(maxP);
    return realMaxPres;

In [71]:
def main(N: int):
    minPresentations = [[] for i in range(1, N+2)];
    minPresentations[0].append(TransversalMatroid(0, [[]]));
    minPresentations[4].append(TransversalMatroid(4, [[1,2],[1,4],[3,4]]));
    minPresentations[4].append(TransversalMatroid(4, [[1,4],[2,4],[3,4]]));
    minPresentations[5].append(TransversalMatroid(5, [[1,5],[2,5],[1,3],[1,4]]));
    minPresentations[5].append(TransversalMatroid(5, [[1,5],[2,5],[1,3],[2,4]]));   
    
    for n in range(N):
        for r, p in enumerate(minPresentations[n]):
            print('n=',n,'i=',r,'out of',len(minPresentations[n])-1)
            r = p.getRank();
            for I in Set(range(len(p.sets))).subsets():
                newMinimal = p.extension(I);
                if not isDuplicate(minPresentations[n+1], newMinimal):
                    minPresentations[n+1].append(newMinimal);
            # Add coloop
            pWithColoop = p.addColoop();
            if not isDuplicate(minPresentations[n+1], pWithColoop):
                minPresentations[n+1].append(pWithColoop);
    return minPresentations

In [73]:
import time;
start_time = time.time()

N = 6 ;
computedIsomorphic = dict();
relabeled = dict();


minPresentations = main(N);
print(time.time() - start_time)
# print("Finished computing minimal presentations in",time.time() - start_time, "seconds");
# print();
mid_time = time.time();
realMaxPres = getMaxPresentations(N, minPresentations);
print(time.time()-mid_time);
print(time.time() - start_time)
# for n in range(1, N+1):
#     print('Maximal presentations for n=',n)
#     for r in range(n+1):
#         print('Transversal matroids of rank',r,'with',n,'elements:',len(realMaxPres[n][r]));

# print("Finished in",time.time() - start_time, "seconds");

n= 0 i= 0 out of 0
n= 1 i= 0 out of 1
n= 1 i= 1 out of 1
n= 2 i= 0 out of 3
n= 2 i= 1 out of 3
n= 2 i= 2 out of 3
n= 2 i= 3 out of 3
n= 3 i= 0 out of 7
n= 3 i= 1 out of 7
n= 3 i= 2 out of 7
n= 3 i= 3 out of 7
n= 3 i= 4 out of 7
n= 3 i= 5 out of 7
n= 3 i= 6 out of 7
n= 3 i= 7 out of 7
n= 4 i= 0 out of 17
n= 4 i= 1 out of 17
n= 4 i= 2 out of 17
n= 4 i= 3 out of 17
n= 4 i= 4 out of 17
n= 4 i= 5 out of 17
n= 4 i= 6 out of 17
n= 4 i= 7 out of 17
n= 4 i= 8 out of 17
n= 4 i= 9 out of 17
n= 4 i= 10 out of 17
n= 4 i= 11 out of 17
n= 4 i= 12 out of 17
n= 4 i= 13 out of 17
n= 4 i= 14 out of 17
n= 4 i= 15 out of 17
n= 4 i= 16 out of 17
n= 4 i= 17 out of 17
n= 5 i= 0 out of 45
n= 5 i= 1 out of 45
n= 5 i= 2 out of 45
n= 5 i= 3 out of 45
n= 5 i= 4 out of 45
n= 5 i= 5 out of 45
n= 5 i= 6 out of 45
n= 5 i= 7 out of 45
n= 5 i= 8 out of 45
n= 5 i= 9 out of 45
n= 5 i= 10 out of 45
n= 5 i= 11 out of 45
n= 5 i= 12 out of 45
n= 5 i= 13 out of 45
n= 5 i= 14 out of 45
n= 5 i= 15 out of 45
n= 5 i= 16 out of 45
